In [1]:

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import os


In [11]:
# chargement des données et localisation du fichier train.csv automatiquement
base_dir = os.path.dirname(os.getcwd())  # monte d’un niveau
csv_path = os.path.join(base_dir, "Data", "processed", "metro_complet.csv")

# chargement du fichier CSV
df = pd.read_csv(csv_path)
df = df[df['Niveau de pollution'] != 'pas de données']

In [ ]:

ligne_selector = widgets.Dropdown(
    options=["Toutes"] + sorted(df["Nom de la ligne"].unique()),
    description='Ligne:',
    value="Toutes"
)

def afficher_carte(ligne):
    if ligne == "Toutes":
        filtered = df
    else:
        filtered = df[df["Nom de la ligne"] == ligne]

    color_discrete_map = {
        "Faible": "vert",
        "Moyen": "orange",
        "ELEVE": "red"
    }

    fig = px.scatter_mapbox(
        filtered,
        lat="stop_lat",
        lon="stop_lon",
        color="Niveau de pollution",
        hover_name="Nom de la Station",
        zoom=11,
        height=500,
        mapbox_style="carto-positron",
        title=f"Stations de métro - {ligne if ligne != 'Toutes' else 'Toutes lignes'}",
        color_discrete_map=color_discrete_map
    )
    fig.show()

widgets.interact(afficher_carte, ligne=ligne_selector)


interactive(children=(Dropdown(description='Ligne:', options=('Toutes', 'Métro 1', 'Métro 10', 'Métro 11', 'Mé…

<function __main__.afficher_carte(ligne)>

In [4]:

pollution_counts = df.groupby(["Nom de la ligne", "niveau_pollution"]).size().reset_index(name='Nombre de stations')
fig = px.bar(pollution_counts, x="Nom de la ligne", y="Nombre de stations", color="niveau_pollution", barmode="group",
             title="Niveau de pollution par ligne de métro")
fig.show()


In [5]:
import pandas as pd
import plotly.graph_objects as go

# Supposons que plusieurs lignes par station existent (on simule l'agrégation ici)
# Regrouper par Nom de la Station et agréger les autres infos en concaténant ou moyennant
grouped = df.groupby("Nom de la Station").agg({
    "Identifiant station": lambda x: ', '.join(set(x)),
    "Nom de la ligne": lambda x: ', '.join(sorted(set(x))),
    "niveau_pollution": lambda x: ', '.join(sorted(set(x))),
    "stop_lon": "mean",
    "stop_lat": "mean"
}).reset_index()

# Création du tableau interactif
fig = go.Figure(data=[go.Table(
    columnwidth=[80]*len(grouped.columns),
    header=dict(values=[f"<b>{col}</b>" for col in grouped.columns],
                fill_color='darkblue',
                font=dict(color='white', size=12),
                align='left'),
    cells=dict(values=[grouped[col] for col in grouped.columns],
               fill_color='lavender',
               align='left',
               font=dict(size=11))
)])

fig.update_layout(
    title="Tableau interactif des stations (regroupées)",
    height=600
)
fig.show()


In [6]:
# Recharger les données
base_dir = os.path.dirname(os.getcwd())
csv_path = os.path.join(base_dir, "Data", "processed", "train.csv")
df = pd.read_csv(csv_path)

# Supprimer les lignes sans info pollution
df = df[df['niveau_pollution'] != 'pas de données']

# Ajouter pollution_num
pollution_mapping = {
    'pollution faible': 1,
    'pollution moyenne': 2,
    'pollution élevée': 3
}
df['pollution_num'] = df['niveau_pollution'].map(pollution_mapping)

# Sauvegarder un nouveau CSV pour QGIS
cols = ['Nom de la Station', 'Nom de la ligne', 'niveau_pollution', 'pollution_num', 'stop_lat', 'stop_lon']
df_export = df[cols]
df_export.to_csv("../Data/processed/stations_qgis.csv", index=False)